In [30]:
!pip install https://github.com/GoogleCloudPlatform/healthcare-api-dicomweb-cli/archive/v1.0.zip

from pathlib import Path
import os

from google.auth.transport import requests
from google.oauth2 import service_account

def dicomweb_search_instance(project_id, location, dataset_id, dicom_store_id):
    """Handles the GET requests specified in DICOMweb standard.

    See https://github.com/GoogleCloudPlatform/python-docs-samples/tree/main/healthcare/api-client/v1/dicom
    before running the sample."""


    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/ubuntu/credentials.json"

    if not Path(os.environ["GOOGLE_APPLICATION_CREDENTIALS"]).exists():
        print("ERROR, CREDENTIALS FILE IS NOT LOCATED")
        print(os.getcwd())
        print(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])

    credentials = service_account.Credentials.from_service_account_file(
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
    )

    scoped_credentials = credentials.with_scopes(
        ["https://www.googleapis.com/auth/cloud-platform"]
    )
    session = requests.AuthorizedSession(scoped_credentials)
    base_url = "https://healthcare.googleapis.com/v1"
    url = f"{base_url}/projects/{project_id}/locations/{location}"
    dicomweb_path = "{}/datasets/{}/dicomStores/{}/dicomWeb/instances".format(
        url, dataset_id, dicom_store_id
    )
    headers = {"Content-Type": "application/dicom+json; charset=utf-8"}

    response = session.get(dicomweb_path, headers=headers)
    response.raise_for_status()
    instances = response.json()

    return instances

project_id = "idc-external-031"
location = "europe-west2"
dataset_id = "kaapana-integration-test"
dicom_store_id = "kaapana-integration-test-store"

all_metadata_dict = dicomweb_search_instance(
    project_id=project_id,
    location=location,
    dataset_id= dataset_id,
    dicom_store_id = dicom_store_id
)

Defaulting to user installation because normal site-packages is not writeable
  Using cached https://github.com/GoogleCloudPlatform/healthcare-api-dicomweb-cli/archive/v1.0.zip
  Preparing metadata (setup.py) ... done


In [39]:
import os
from google.auth.transport import requests
from google.oauth2 import service_account

In [42]:
series_uid = '1.3.6.1.4.1.14519.5.2.1.99.1071.15503385522001218410281775124839'

In [43]:
base_url = "https://healthcare.googleapis.com/v1/projects/idc-external-031/locations/europe-west2/datasets/kaapana-integration-test/dicomStores/kaapana-integration-test-store/dicomWeb"

In [44]:
dicomweb_endpoint = "https://healthcare.googleapis.com/v1/projects/idc-external-031/locations/europe-west2/datasets/kaapana-integration-test/dicomStores/kaapana-integration-test-store/dicomWeb"

In [58]:
def get_instances_of_series(series_uid):
    credentials = service_account.Credentials.from_service_account_file(
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
    )
    scoped_credentials = credentials.with_scopes(
        ["https://www.googleapis.com/auth/cloud-platform"]
    )
    session = requests.AuthorizedSession(scoped_credentials)

    series_path = f"{dicomweb_endpoint}/instances"
    params = {"SeriesInstanceUID": series_uid}

    # Set the required Accept header on the request
    headers = {"Content-Type": "application/dicom+json; charset=utf-8"}
    response = session.get(series_path, headers=headers)

    response = response.json()
    instance_list = []
    for instance in response:
        instance_list.append(
            [
                instance["0020000D"]["Value"][0],  # StudyInstanceUID
                instance["0020000E"]["Value"][0],  # SeriesInstanceUID
                instance["00080018"]["Value"][0],  # SOPInstanceUID
            ]
        )

In [ ]:
def get_session():
    credentials = service_account.Credentials.from_service_account_file(
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
    )
    scoped_credentials = credentials.with_scopes(
        ["https://www.googleapis.com/auth/cloud-platform"]
    )
    session = requests.AuthorizedSession(scoped_credentials)
    return session

In [46]:
def downloadSeries(series_uid):
    dicomwebpath = "https://healthcare.googleapis.com/v1/projects/idc-external-031/locations/europe-west2/datasets/kaapana-integration-test/dicomStores/kaapana-integration-test-store/dicomWeb"

    instance_list = get_instances_of_series(series_uid)

    for instance in instance_list:
        dicomweb_path = f"{base_url}/studies/{instance[0]}/series/{instance[1]}/instances/{instance[2]}"

    file_name = "instance.dcm"

    # Set the required Accept header on the request
    headers = {"Accept": "application/dicom; transfer-syntax=*"}
    response = session.get(dicomweb_path, headers=headers)
    response.raise_for_status()

    with open(file_name, "wb") as f:
        f.write(response.content)
        print(
            "Retrieved DICOM instance and saved to {} in current directory".format(
                file_name
            )
        )

In [59]:
downloadSeries(series_uid)

TypeError: 'NoneType' object is not iterable

In [51]:
from dicomweb_client.api import DICOMwebClient
from dicomweb_client.ext.gcp.session_utils import create_session_from_gcp_credentials
from dicomweb_client.ext.gcp.uri import GoogleCloudHealthcareURL

session = create_session_from_gcp_credentials()

url = GoogleCloudHealthcareURL(
    project_id='idc-external-031',
    location='europe-west2',
    dataset_id='kaapana-integration-test',
    dicom_store_id='kaapana-integration-test-store'
)

client = DICOMwebClient(
    url=str(url),
    session=session
)

In [55]:
studies = client.search_for_studies()

In [56]:
studies

[{'00080005': {'vr': 'CS', 'Value': ['ISO_IR 100']},
  '00080020': {'vr': 'DA', 'Value': ['19001016']},
  '00080030': {'vr': 'TM', 'Value': ['120024.000']},
  '00080050': {'vr': 'SH'},
  '00080090': {'vr': 'PN'},
  '00100010': {'vr': 'PN', 'Value': [{'Alphabetic': 'A951074'}]},
  '00100020': {'vr': 'LO', 'Value': ['A951074']},
  '00100030': {'vr': 'DA'},
  '00100040': {'vr': 'CS', 'Value': ['M']},
  '0020000D': {'vr': 'UI',
   'Value': ['1.3.6.1.4.1.14519.5.2.1.99.1071.15503385522001218410281775124839']},
  '00200010': {'vr': 'SH'}}]

In [57]:
frames = client.retrieve_instance_frames_rendered(
    study_instance_uid='1.2.826.0.1.3680043.8.1055.1.20111103111148288.98361414.79379639',
    series_instance_uid='1.2.826.0.1.3680043.8.1055.1.20111103111208937.49685336.24517034',
    sop_instance_uid='1.2.826.0.1.3680043.8.1055.1.20111103111208937.40440871.13152534',
    frame_numbers=[1],
    media_types=('image/jpeg', ),
    params={'quality': 95, 'iccprofile': 'yes'}
)

HTTPError: 400 Client Error: Bad Request for url: https://healthcare.googleapis.com/v1/projects/idc-external-031/locations/europe-west2/datasets/kaapana-integration-test/dicomStores/kaapana-integration-test-store/dicomWeb/studies/1.2.826.0.1.3680043.8.1055.1.20111103111148288.98361414.79379639/series/1.2.826.0.1.3680043.8.1055.1.20111103111208937.49685336.24517034/instances/1.2.826.0.1.3680043.8.1055.1.20111103111208937.40440871.13152534/frames/1/rendered?quality=95&iccprofile=yes